In [1]:
import numpy
import random
import math
import time


### NONVOLATILE STATUS CLASS
 

class Status:
    def __init__(self, name, min_turns = 1, max_turns = numpy.inf, healingprob = 0):
        self.name = name
        self.min_turns = 1
        self.max_turns = numpy.inf
        self.healingprob = healingprob
        
    def __repr__(self):
        return self.name
    
    def heal(self, pokemon, turn):
        pass
    
    
class BaseStatus(Status):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def mid_turn_effect(self,pokemon):
        return True
    
    def end_turn_effect(self,pokemon):
        pass
    
            
class Vol(Status):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def heal(self, pokemon, turn):
        check = random.choices([True, False],[self.healingprob,1-self.healingprob])[0]
        if turn > self.max_turns or turn > self.min_turns and check:
            print('\n{} is not {} anymore.\n'.format(pokemon, self))
            pokemon.vol_statuses.pop(self)
            
class Confuse(Vol):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def mid_turn_effect(self, pokemon, inlove):
        print('\n{} is confused.'.format(pokemon))
        check = random.choices([True, False],[0.5,0.5])[0]
        if not check:
            print('\nIt hurts himself in his confusion!\n')
            conf = AttackMove(40, 'Conf', Standard, 1, precision = None, nonmod = True)
            conf.effect(pokemon, pokemon)
        return check
    
    def end_turn_effect(self, pokemon):
        pass
    
class Cursing(Vol):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def mid_turn_effect(self, pokemon, inlove):
        return True
    
    def end_turn_effect(self, pokemon):
        print("\nCurse hits {}\n".format(pokemon))
        pokemon.lp_adjourn(-round((pokemon.skillset[Health])/4))
    
class Attract(Vol):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def heal(self, pokemon, turn):
        for player in pokemon.battle.players:
            if player != pokemon.trainer:
                opponent = player.team[0]
                
        if pokemon.inlove != opponent:
            pokemon.vol_statuses.pop(self)
            return
        
        super().heal(pokemon, turn)
        
    def mid_turn_effect(self, pokemon, inlove):
        print('\n{} is in love with {}.\n'.format(pokemon, inlove))
        check = random.choices([True, False],[0.5,0.5])[0]
        if not check:
            print("{} is not able to attack {} because of his attraction!\n".format(pokemon, inlove))
        return check  
    
    def end_turn_effect(self, pokemon):
        pass
    
    
    
class NonVol(Status):
    def __init__(self, *args, immune_types = [], affected_stats = {}, **kwargs):
        super().__init__(*args, **kwargs)
        self.immune_types = immune_types.copy()
        self.affected_stats = affected_stats.copy()
        
    def heal(self, pokemon, turn):
        check = random.choices([True, False],[self.healingprob,1-self.healingprob])[0]
        if turn > self.max_turns or (turn > self.min_turns and type(self) != BaseStatus and check):
            for stat, affect in self.affected_stats.items():
                pokemon.battle_skillset[stat] /= affect
            print('{} is not {} anymore.'.format(pokemon, self))                                        
            pokemon.nonvol = Base   
            pokemon.nonvolturns = 1
    
class DamageStatus(NonVol):
    def __init__(self, *args, strenght = 1/16, increment = 1, **kwargs):
        super().__init__(*args, **kwargs)
        self.strenght = strenght
        self.increment = increment
        
    def mid_turn_effect(self,pokemon):
        return True
            
    def end_turn_effect(self, pokemon):
        print('\n{} is {}!\n'.format(pokemon, self))
        pokemon.lp_adjourn(-round((self.increment**(pokemon.nonvolturns-1))*self.strenght*pokemon.skillset[Health]))
        pokemon.nonvolturns += 1 
    
class BlockStatus(NonVol):
    def __init__(self, *args, blockprob = 1, **kwargs):
        super().__init__(*args, **kwargs)
        self.blockprob = blockprob
        
    def mid_turn_effect(self, pokemon):
        check = random.choices([False,True],[self.blockprob,1-self.blockprob])[0]
        if not check:
            print('\n{} is {}.\n'.format(pokemon, self))
        return check
    
    def end_turn_effect(self, pokemon):
        pass
    
    

### SPECIES CLASS



class Species:
    def __init__(self, name, types, base_skillset, propension, immune_status = []):
        self.name = name
        self.types = types
        self.base_skillset = base_skillset
        self.immune_status = immune_status.copy()
        self.propension = propension

    def __repr__(self):
        features = sorted(self.base_skillset, key = self.base_skillset.get)
        best_feat, worst_feat = features[-2].name, features[0].name
        types = [str(typ) for typ in self.types]
        if len(types) > 1:
            return "Pokemon Species: {} of Types: "+len(types)*"{}"+". Good on {} and Bad on {}".format(self.name, *types, best_feat.upper(),worst_feat.upper())
        return "Pokémon Species: {} of Type: {}. Good on {} and Bad on {}".format(self.name,types[0].upper(),best_feat.upper(),worst_feat.upper())



### POKEMON CLASS

    
    
class Pokemon: 
    def __init__(self, species, pkmnname, level, sex, moveset, nature, multipliers = None, trainer = None, skillset=None, battle_skillset = None, lp=None, confused = False, status=None, item_held=None, nonvol = BaseStatus('OK'), nonvolturns = 1, vol_statuses = {}, inlove = None, battle = None):
        self.species = species
        self.trainer = trainer
        if self.trainer is not None:
            self.battle = trainer.battle
        self.name = pkmnname
        self.level = level
        self.sex = sex
        self.moveset = moveset
        if skillset is None:
            self.skillset = self.species.base_skillset.copy()
        else:
            self.skillset = skillset.copy()
        self.battle_skillset = battle_skillset
        if self.battle_skillset is None:
            self.battle_skillset = self.skillset.copy()
            self.battle_skillset[Health]
        self.stat_mult = multipliers
        if self.stat_mult is None:
            self.stat_mult = {key:0 for key in self.battle_skillset}
        self.lp = lp
        self.nature = nature
        self.confused = confused
        self.nonvol = BaseStatus('OK')
        if type(nonvol) != BaseStatus:
            self.nonvol_status_adjourn(nonvol, beginning = True)
        self.nonvolturns = nonvolturns
        self.vol_statuses = vol_statuses.copy()
        self.inlove = inlove
        self.item_held = item_held
        if skillset is None:
            self.skillset = self.species.base_skillset
        if lp is None:
            self.lp = self.species.base_skillset[Health]
        self.ko = self.lp <= 0
    
    def __repr__(self):
        return self.name
    
    def moveset_display(self):
        s = ''
        for i, elem in enumerate(self.moveset):
            s += ' | {} = {} \n'.format(i+1, elem)
        for i in range(max(0, 4-len(self.moveset))):
            s += ' | ////  \n'
        return s
        
    def display_own(self):
        print()
        print('-- {} is currently fighting. --'.format(self))
        print('-- hp: {}/{} --'.format(self.lp, self.skillset[Health]))
        print('-- status: {}.'.format(self.nonvol))
        print()
        
    def display_opponent(self):
        print()
        print('-- {} is the current opponent. --'.format(self))
        perc = 10*math.ceil(10*(self.lp/self.skillset[Health]))
        if perc == 100:
            print('-- Remaining hp: Full Energy (100%) --')
            print('-- status: {}.'.format(self.nonvol))
            return 
        print('-- Remaining hp between {}% and {}% --'.format(perc,perc-10))
        print('-- status: {}.'.format(self.nonvol))
        print()
        
    def lp_adjourn(self, delta):
        opponent = self.battle.players[self.battle.players.index(self.trainer)-1]
        opponents = [opp for opp in opponent.team if not opp.ko]
        self.lp = max(0,min(self.skillset[Health], self.lp+delta))
        self.ko = self.lp <= 0
        if self.ko:
            print('\n{} is exhausted.\n'.format(self.name))
        self.battle.check_end()
        if self.battle.over and self.ko and len(opponents) > 0:
            print('\n{} wins.'.format(opponent))
        return 
    
    def stat_adjourn(self, adjournments): # adjournments = {stat: incr}, es: {attack:2, defense:-2}
        for stat, variation in adjournments.items():                
            old = self.stat_mult[stat]
            self.stat_mult[stat] = max(-6,min(6,old + variation))
            if variation > 0:
                sign = 'maximum'
                action = 'rose'
            else:
                sign = 'minimum'
                action = 'fell'
            if old == self.stat_mult[stat]:
                
                print("{}'s {} is already at {}.".format(self,stat,sign))
                return 
            
            print("{}'s {} {}".format(self, stat, action))
            
            self.battle_skillset[stat] = stat.multipliers[self.stat_mult[stat]]*self.skillset[stat]
    
    def nonvol_status_adjourn(self, new_val, beginning = False):
        if type(new_val) != BaseStatus:
            if type(self.nonvol) != BaseStatus:
                print('{} is already {}.'.format(self, self.nonvol))
                return 
            
            if new_val in self.species.immune_status:
                print("{} can't be {}.".format(self,new_val))
                return
            
            for typ in self.species.types:
                if typ in new_val.immune_types:
                    print("{} Pokémons can't be {}.".format(typ, new_val))
                    return
                
            self.nonvol = new_val
            if not beginning:
                print('{} is now {}.'.format(self,self.nonvol))
            for stat, affect in self.nonvol.affected_stats.items():
                self.battle_skillset[stat] *= affect
            return    
        
        if type(self.nonvol) == BaseStatus:
            print("{} is already fine".format(self))
            return 
        
        self.nonvol = BaseStatus('OK') 
    
    def vol_status_adjourn(self, new_status, attacker):
        addition = ''
        if type(new_status) == Attract:
            self.inlove = attacker
            addition = ' to {}'.format(self.inlove)
        
        if new_status not in self.vol_statuses.keys():
            print('{} is now {}{}.'.format(self, new_status, addition))
            self.vol_statuses[new_status] = 1
        
        
    
### STAT CLASS
    
    
    
class Stat:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return self.name
    
    
class Accuracy(Stat):
    multipliers = {-6:1/3, -5:3/8, -4:3/7, -3:0.5, -2:3/5, -1:0.75, 0:1, 1:4/3, 2:5/3, 3:2, 4:7/3, 5:8/3, 6:3}

    
    
class Fighting(Stat):
    multipliers = {-6:0.25, -5:2/7, -4:1/3, -3:0.4, -2:0.5, -1:2/3, 0:1, 1:1.5, 2:2, 3:2.5, 4:3, 5:3.5, 6:4}
            
            
            
### MOVETYPE CLASS

            
            
class MoveType:
    def __init__(self, name, typ, base_pp, precision = 1, max_pp = numpy.inf, immune = [], nonmod = False):
        self.name = name
        self.type = typ
        self.precision = precision
        self.base_pp = base_pp      
        self.max_pp = max_pp
        self.immune = immune.copy()
        self.nonmod = nonmod
        
    def __repr__(self):
        return self.name
    
    def can_attack(self, defender):
        for typ in defender.species.types:
            if self.type in typ.immune:
                print('{} Pokémons are immune to moves of {} type'.format(defender, self.type))
                return False
            if typ in self.immune:
                print('{} is immune to this move.'.format(defender))
                return False
        return True
    
    def precision_calc(self, attacker, defender):
        if self.precision is not None:
            prob = min(1,max(0,self.precision*attacker.battle_skillset[Precision]/defender.battle_skillset[Evasion]))
            if not random.choices([True, False],[prob,1-prob])[0]:
                print('{} misses!'.format(attacker))
                return False
        return True 
        
        
class AttackMove(MoveType):
    def __init__(self, strenght,*args, special = False, recoil = 0, recoil_stat = 'dmg', recoil_msg = 'suffers for the recoil', autostrenght = 0, **kwargs):
        super().__init__(*args, **kwargs)
        self.specific(strenght, special, recoil, recoil_stat, recoil_msg, autostrenght)
        
    def specific(self, strenght, special, recoil, recoil_stat, recoil_msg, autostrenght):
        self.strenght = strenght
        self.special = special
        self.recoil = recoil
        self.recoil_stat = recoil_stat
        self.autostrenght = autostrenght
        self.recoil_msg = recoil_msg
        
    def effect(self, attacker, defender):
        if self.autostrenght != 0:
            if self.special:
                df = attacker.battle_skillset[Def_Spec]
            else:
                df = attacker.battle_skillset[Def]
                autodamage = math.ceil(((((2*attacker.level*mod+10)*att*self.strenght)/(250*df))+2)*mult*stab*condition*N)
            if autodamage > 0:
                print('Auto Damage: ', autodamage)
            elif autodamage < 0:
                print('Auto Recovery: ', autodamage)
                    
            attacker.lp_adjourn(-autodamage)
        damage = 0
        
        if self.can_attack(defender) and self.strenght is not None:
            mult = numpy.prod([self.type.atk_strenght.get(typ,1) for typ in defender.species.types])
            stab = 1
            weights = [0.9,0.1]
            if self.nonmod:
                weights = [1,0]
            mod = random.choices([1,2],weights)[0]
            condition = 1 # Da aggiornare
            if self.type in attacker.species.types:
                stab *= 1.5
                
            if mult > 1:
                print("It's super-effective!")
            elif mult < 1:
                print("It's not very effective..")
        
            if mod == 2:
                print('Critical hit!')
                
            if self.special:
                att, df = attacker.battle_skillset[Atk_Spec], defender.battle_skillset[Def_Spec]
            else:
                att, df = attacker.battle_skillset[Atk], defender.battle_skillset[Def]
            
            N = random.choice(range(85,101))/100
            
            damage = math.ceil(((((2*attacker.level*mod+10)*att*self.strenght)/(250*df))+2)*mult*stab*condition*N)
                
            if damage > 0:
                print('Damage: ', damage)
                
            elif damage < 0:
                print('Recovery: ', damage)
                
            defender.lp_adjourn(-damage)
            
        if self.recoil > 0:
            print('\n{} {}!\n'.format(attacker, self.recoil_msg))
            if self.recoil_stat == 'dmg':
                recoil = round(self.recoil*damage)
            elif self.recoil_stat == 'hp':
                recoil = round(self.recoil*attacker.skillset[Health])
            else:
                recoil = 0
            attacker.lp_adjourn(-recoil)
                
        
class StatMove(MoveType):
    def __init__(self, adjournments, *args, autoadjournments = {}, **kwargs):
        super().__init__(*args,**kwargs)
        self.specific(adjournments, autoadjournments)
        
    def specific(self, adjournments, autoadjournments):
        self.adjournments = adjournments
        self.autoadjournments = autoadjournments.copy()
        
    def effect(self, attacker, defender):
        defender.stat_adjourn(self.adjournments)
        if self.autoadjournments != {}:
            attacker.stat_adjourn(self.autoadjournments)
        
class NonVolMove(MoveType):
    def __init__(self, new_nonvol, *args, auto_nonvol = None, **kwargs):
        super().__init__(*args,**kwargs)
        self.specific(new_nonvol, auto_nonvol)
        
    def specific(self, new_nonvol, auto_nonvol):
        self.new_nonvol = new_nonvol
        self.auto_nonvol = auto_nonvol
        
    def effect(self, attacker, defender):
        defender.nonvol_status_adjourn(self.new_nonvol)
        if self.auto_nonvol is not None:
            attacker.nonvol_status_adjourn(self.auto_nonvol)
        
class VolMove(MoveType):
    def __init__(self, new_vol, *args, auto_vol = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.specific(new_vol, auto_vol)
        
    def specific(self, new_vol, auto_vol):
        self.new_vol = new_vol
        self.auto_vol = auto_vol
    
    def effect(self, attacker, defender):
        defender.vol_status_adjourn(self.new_vol, attacker)
        if self.auto_vol is not None:
            attacker.vol_status_adjourn(self.auto_vol, attacker)

        
class AttackStat(AttackMove, StatMove):
    def __init__(self, strenght, adjournments, *args, autoadjournments = {}, special = False, recoil = 0, recoil_stat = 'dmg', recoil_msg = 'suffers for the recoil', autostrenght = 0, **kwargs):
        MoveType.__init__(self, *args, **kwargs)
        AttackMove.specific(self, strenght, special = special, recoil = recoil, recoil_stat = recoil_stat, recoil_msg = recoil_msg, autostrenght = autostrenght)
        StatMove.specific(self, adjournments, autoadjournments)
        
    def effect(self, attacker, defender):
        AttackMove.effect(self, attacker, defender)
        StatMove.effect(self, attacker, defender)
        
class AttackNonVol(AttackMove, NonVolMove):
    def __init__(self, strenght, new_nonvol, *args, auto_nonvol = None, special = False, recoil = 0, recoil_stat = 'dmg', recoil_msg = 'suffers for the recoil', autostrenght = 0, **kwargs):
        MoveType.__init__(self, *args, **kwargs)
        AttackMove.specific(self, strenght, special = special, recoil = recoil, recoil_stat = recoil_stat, recoil_msg = recoil_msg, autostrenght = autostrenght)
        NonVolMove.specific(self, new_nonvol, auto_nonvol)
        
    def effect(self, attacker, defender):
        AttackMove.effect(self, attacker, defender)
        NonVolMove.effect(self, attacker, defender)
        
class AttackVol(AttackMove, VolMove):
    def __init__(self, strenght, new_vol, *args, auto_vol = None, special = False, recoil = 0, recoil_stat = 'dmg', recoil_msg = 'suffers for the recoil', autostrenght = 0, **kwargs):
        MoveType.__init__(self, *args, **kwargs)
        AttackMove.specific(self, strenght, special = special, recoil = recoil, recoil_stat = recoil_stat, recoil_msg = recoil_msg, autostrenght = autostrenght)
        VolMove.specific(self, new_vol, auto_vol)
        
    def effect(self, attacker, defender):
        AttackMove.effect(self, attacker, defender)
        VolMove.effect(self, attacker, defender)
        
        
        
### MOVE CLASS



class Move:
    def __init__(self, movetype, pptot=None, pp=None):
        self.movetype = movetype
        self.pptot = pptot
        if self.pptot is None:
            self.pptot = self.movetype.base_pp
        self.pp = pp
        if self.pp is None:
            self.pp=self.pptot
            
    def __repr__(self):
        return '{} ({}pp/{}pp)'.format(self.movetype,self.pp,self.pptot)



#TYPE CLASS



class Type:
    def __init__(self, name, strong, weak, atk_strenght = {}, immune = []):
        self.name = name
        self.strong = strong
        self.weak = weak
        self.immune = immune.copy()
        self.atk_strenght = atk_strenght.copy()
        for typ in self.strong:
            self.atk_strenght[typ] = 2
        for typ in self.weak:
            self.atk_strenght[typ] = 0.5
            
    def __repr__(self):
        return self.name
            
    def add_strenght(self,strenght):
        self.strong.append(strenght)
        self.atk_strenght[strenght] = 2
    
    def add_weakness(self, weakness):
        self.weak.append(weakness)
        self.atk_strenght[weakness] = 0.5

        
        
### EFFECT CLASS
        
        
        
class Effect:
    def __init__(self, name, effect_prob = 1, progression = 0):
        self.name = name
        self.effect_prob = effect_prob
        self.progression = 0
        
        
        
### ITEM CLASS



class Items:
    def __init__(self,name,lpgain=0,status_affected=None,ppgain=0,move=None):
        self.name = name
        self.lpgain = lpgain
        self.status = status_affected
        self.ppgain = ppgain
        self.move = move
        

        
### Player CLASS



class Player:
    def __init__(self, name, team, bag, fighting = True, battle = None):
        self.name = name
        self.team = team
        self.bag = bag
        self.fighting = fighting
        self.battle = battle
        
    def __repr__(self):
        return self.name
    
    def team_display(self):
        s = ''
        for i, pokemon in enumerate(self.team):
            c = ''
            if pokemon.ko: 
                c = 'EXHAUSTED'
            s += ' | {} = {} {}\n'.format(i+1, pokemon, c)
        for i in range(max(0,6-len(self.team))):
            s += ' | //// \n'
        return s
        
    def attack(self,action_obj):
        attacker = self.team[0]
        moves = attacker.moveset
        if len(list(filter(lambda x: x.pp > 0, attacker.moveset))) == 0:
            move = Move(Struggle)
            return
        move = self.input_choice(moves, attacker.moveset_display())
        while move != 'BACK' and move.pp < 1:
            print('You have no pp left for this move.')
            move = self.input_choice(moves, attacker.moveset_display())
        if move == 'BACK':
            action_obj.back = True
        action_obj.move = move
        action_obj.attacker = attacker
        
    def use(self, action_obj):
        items = self.bag.items
        item = self.input_choice(items, self.bag_display())
        pokemon == 'BACK'
        while pokemon == 'BACK':
            item = self.input_choice(items)
            pokemon = self.input_choice(self.team)
        
        if item == 'BACK':
            action_obj.back = True
        
        action_obj.item = item
        action_obj.pokemon = pokemon
        
    def change(self, action_obj):
        print(self.team_display())
        pokemon = self.input_choice(self.team, self.team_display())
        if pokemon == 'BACK':
            action_obj.back = True
        action_obj.new_pkmn = pokemon
        
    def run(self,action_obj):
        action_obj.escape = True
        
    def input_choice(self, object_list, msg_display = ''):
        opt = None
        strlis = [obj for obj in object_list if type(obj) == str]
        if object_list == [] or len(strlis) == len(object_list):
            print('This section is empty')
            self.back()
        obj_type = str(type(object_list[0]))
        
        list_with_back = ['BACK'] + object_list
        ind = -1
        while opt is None:
            input_msg = 'Choose {}: \n'.format(obj_type[obj_type.find('.')+1:-2])
            input_msg += msg_display
            input_msg += '\n | 0 = BACK \n'.format(len(list_with_back))
            ind = input((input_msg+'\n'))
            try:
                opt = list_with_back[int(ind)]
            except:
                print('\nChoose an Available Option.')
        return opt
    
    def add_pkmn_to_team(self, pokemon):
        if len(self.team) < 6:
            self.team.append(pokemon)

    
    
### ACTION CLASSES



class Action:
    def __init__(self, player, back = False,**kwargs):
        self.player = player
        self.back = back
        self.__dict__.update(kwargs)
        
class Attack(Action):
    priority = 0
    def __init__(self, player, **kwargs):
        self.speed = player.team[0].battle_skillset[Speed]
        player.attack(self)
        super().__init__(player, **kwargs)
        
    def apply(self):
        for train in self.turn.players:
            if not train.team[0] == self.attacker:
                defender = train.team[0]
        for status in self.attacker.vol_statuses.keys():
            status.heal(self.attacker, self.attacker.vol_statuses[status])
        self.attacker.nonvol.heal(self.attacker, self.attacker.nonvolturns)
        if self.attacker.nonvol.mid_turn_effect(self.attacker):
            if self.move.movetype.precision_calc(self.attacker, defender):
                checks = [vol.mid_turn_effect(self.attacker, defender) for vol in self.attacker.vol_statuses.keys()]
                if checks.count(True) == len(self.attacker.vol_statuses):
                    print('\n{} uses {}.\n'.format(self.attacker, self.move.movetype))
                    self.move.movetype.effect(self.attacker, defender)
                    self.move.pp -= 1
                self.turn.battle.check_end()

class Run(Action):
    priority = 2
    def __init__(self, player, **kwargs):
        player.run(self)
        super().__init__(player, kwargs)
        
    def apply(self):
        #Verifica possibilità 
        self.turn.battle.over = True
        
class Change(Action):
    priority = 1
    def __init__(self,player,**kwargs):
        player.change(self)
        super().__init__(player, kwargs)
        
    def apply(self):
        self.player.team.remove(self.pokemon)
        self.player.team.insert(0, self.pokemon)
        
class Use(Action):
    priority = 1
    def __init__(self,player,**kwargs):
        player.use(self)
        super().__init__(player,kwargs)
        
    def apply(self):
        self.item.apply_effect(self.pokemon)

        
        
### BAG CLASS



class Bag:
    def __init__(self, items):
        self.items = items

        
        
### TURN CLASS



class Turn:
    action_dict = {'F': Attack, 'B': Use, 'P': Change, 'R': Run}
    def __init__(self, battle):
        self.battle = battle
        self.players = battle.players
        
    def actions_setup(self):
        actions = {self.players[0]: Action(self.players[0], back = True), self.players[1]: Action(self.players[1], back = True)}
        for player in self.players:
            while actions[player].back:
                actions[player] = self.action_dict[self.action_choice(player)](player, turn = self)
            print('\n\n\n')
        self.actions = actions
    
    def action_choice(self, player):
        print("-----------------  {}'s turn.  -----------------".format(player))
        player.team[0].display_own()
        for play in self.players:
            if play != player:
                play.team[0].display_opponent()
        action = 'Z'
        while action[0].upper() not in ('B','F','P','R'):
            action = input('Bag (B), Fight (F), Pokémon (P), Run (R) \n\n')
            if action[0].upper() not in ('B', 'F', 'P', 'R'):
                print('Choose an available action.')
        return action.upper()[0]
    
    def establish_priorities(self):
        max_pr = -numpy.inf
        player = None
        for elem in self.players:
            if len(list(filter(lambda x: type(x) == Attack, list(self.actions.values())))) == 2:
                if self.actions[elem].speed >= max_pr:
                    player = elem
                    max_pr = self.actions[elem].speed
            else:
                if self.actions[elem].priority >= max_pr:
                    player = elem
                    max_pr = self.actions[elem].priority
                    
        self.players.remove(player)
        self.players.insert(0, player)        
        
    
    def play_turn(self):
        self.establish_priorities()
        for player in self.players:
            opponent = self.players[self.players.index(player)-1]
            self.actions[player].apply()
            if self.battle.over:
                return
            print(opponent.team[0], opponent.team[0].lp)
            if opponent.team[0].ko:
                new_pkmn = Change(opponent, back = True, turn = self)
                while new_pkmn.back:
                    print('You must use a new pokemon.')
                    new_pkmn.prepare()
                new_pkmn.apply()
                 
        for player in self.players:
            player.team[0].nonvol.end_turn_effect(player.team[0])
            if self.battle.over:
                return
    
            for vol in player.team[0].vol_statuses:
                vol.end_turn_effect(player.team[0])
                if self.battle.over:
                    return
                
                player.team[0].vol_statuses[vol] += 1

            
                                
### BATTLE CLASS



class Battle:
    def __init__(self, players, turn=None, over = False):
        self.turn = turn
        self.players = players
        self.wild = 'Wild' in self.players
        self.over = over
        self.initialize_players()
        
    def initialize_players(self):
        for player in self.players:
            player.fighting = True
            player.battle = self
            for pokemon in list(filter(lambda x: type(x) == Pokemon, player.team)):
                pokemon.battle = self
                
    def check_end(self):
        for i in range(2):
            defeated = True
            pkmnlist = [pkmn for pkmn in self.players[i].team if type(pkmn) == Pokemon]
            for pkmn in pkmnlist:
                if not pkmn.ko:
                    defeated = False
                    break
            if defeated:
                self.over = True
                break
                
    def play(self):
        while not self.over:
            turn = Turn(self)
            print('\n-----------------------------------------------------------\n')
            print('                       SETUP PHASE                             ')
            print('\n-----------------------------------------------------------\n')
            turn.actions_setup()
            time.sleep(1)
            print('\n-----------------------------------------------------------\n')
            print('                       FIGHT PHASE                             ')
            print('\n-----------------------------------------------------------\n')
            turn.play_turn()
            time.sleep(3)

In [ ]:
Standard = Type('Standard',[],[])
Normal = Type('Normal',[],[])
Fire = Type('Fire',[],[])
Water = Type('Water',[Fire],[])
Leaf = Type('Leaf',[Water],[Fire])
Fire.add_strenght(Leaf)
Fire.add_weakness(Water)
Water.add_weakness(Leaf)

Health = Fighting('hp')
Atk = Fighting('attack')
Def = Fighting('defense')
Atk_Spec = Fighting('attack special')
Def_Spec = Fighting('defense special')
Speed = Fighting('speed')
Precision = Accuracy('precision')
Evasion = Accuracy('evasion')

Base = BaseStatus('fit')
Paralyzed = BlockStatus('paralyzed', affected_stats = {Speed:0.5}, blockprob = 0.25)
Asleep = BlockStatus('asleep', max_turns = 5, healingprob = 0.2)
Freeze = BlockStatus('frozen', max_turns = 5, healingprob = 0.2)
Poison = DamageStatus('poisoned')
Bad_Poison = DamageStatus('bad poisoned', increment = 2)
Burn = DamageStatus('burn')

Confused = Confuse('confused', min_turns = 2, max_turns = 5, healingprob = 0.2)
Attracted = Attract('attracted', min_turns = 2, max_turns = 5, healingprob = 0.2)
Cursed = Cursing('cursed')


Struggle = AttackMove(50, 'Struggle', Normal, 1, recoil = 0.25)
Scratch = AttackMove(40, 'Scratch',Normal, 35)
Tackle = AttackMove(40, 'Tackle',Normal, 35)

Harden = StatMove({}, 'Harden', Normal, 30, max_pp = 48, autoadjournments = {Def:1}, precision = None)
Growl = StatMove({Atk:-1}, 'Growl', Normal, 40, max_pp = 64)
Tail_Whip = StatMove({Def:-1}, 'Tail Whip', Normal, 40, max_pp = 64)

Thunder_Wave = NonVolMove(Paralyzed, 'Thunder Wave', Normal, 20, max_pp = 32, precision = 0.9)

Attraction = VolMove(Attracted, 'Attract', Normal, 15, max_pp = 24)
Curse = AttackVol(None, Cursed, 'Curse', Normal, 10, max_pp = 16, precision = None, recoil = 0.5, recoil_stat = 'hp', recoil_msg = 'loses half of its hp')

Charmander = Species('Charmender',[Fire],{Health:19, Atk:11, Def:10, Atk_Spec:11, Def_Spec:10, Speed:12, Precision:1, Evasion:1}, {Health:0.4, Atk:0.6, Def:0.5, Atk_Spec:0.6, Def_Spec:0.5, Speed:0.7})
Bulbasaur = Species('Bulbasaur',[Leaf],{Health:20, Atk:10, Def:10, Atk_Spec:12, Def_Spec:12, Speed:10, Precision:1, Evasion:1},{Health:0.5, Atk:0.5, Def:0.5, Atk_Spec:0.7, Def_Spec:0.7, Speed:0.4})
Squirtle = Species('Squirtle',[Water],{Health:20, Atk:10, Def:12, Atk_Spec:10, Def_Spec:12, Speed:10, Precision:1, Evasion:1},{Health:0.5, Atk:0.5, Def:0.7, Atk_Spec:0.5, Def_Spec:0.7, Speed:0.4})

Ash = Player('Ash',[],[])
Gary = Player('Gary',[],[])

charmander = Pokemon(Charmander,'Charmander',5,'M',[Move(Scratch), Move(Growl), Move(Harden), Move(Curse)], 'Calm', trainer = Ash)
bulbasaur = Pokemon(Bulbasaur, 'Bulbasaur', 5, 'M',[Move(Tackle), Move(Tail_Whip)],'Curious')
squirtle = Pokemon(Squirtle,'Squirtle', 5, 'F',[Move(Tackle), Move(Growl), Move(Thunder_Wave), Move(Struggle)],'Lively', trainer = Gary)

Ash.add_pkmn_to_team(charmander)
Gary.add_pkmn_to_team(squirtle)
Gary.add_pkmn_to_team(bulbasaur)

Battle([Ash, Gary],None).play()


-----------------------------------------------------------

                       SETUP PHASE                             

-----------------------------------------------------------

-----------------  Ash's turn.  -----------------

-- Charmander is currently fighting. --
-- hp: 19/19 --
-- status: OK.


-- Squirtle is the current opponent. --
-- Remaining hp: Full Energy (100%) --
-- status: OK.
Bag (B), Fight (F), Pokémon (P), Run (R) 

F
Choose Move: 
 | 1 = Scratch (35pp/35pp) 
 | 2 = Growl (40pp/40pp) 
 | 3 = Harden (30pp/30pp) 
 | 4 = Curse (10pp/10pp) 

 | 0 = BACK 

4




-----------------  Gary's turn.  -----------------

-- Squirtle is currently fighting. --
-- hp: 20/20 --
-- status: OK.


-- Charmander is the current opponent. --
-- Remaining hp: Full Energy (100%) --
-- status: OK.
Bag (B), Fight (F), Pokémon (P), Run (R) 

F
Choose Move: 
 | 1 = Tackle (35pp/35pp) 
 | 2 = Growl (40pp/40pp) 
 | 3 = Thunder Wave (20pp/20pp) 
 | 4 = Struggle (1pp/1pp) 

 | 0 = BACK 

2